In [1]:
import torch
import matplotlib.pyplot as plt
import dlc_practical_prologue as prologue
from torch import nn

#from torch.autograd import Variable
#from torch import nn
from torch.nn import functional as F

## Generating the dataset

In [2]:
nbr_pairs = 1000
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(nbr_pairs)

print('train_input size =', train_input.size())
print('train_target size =', train_target.size()) #The boolean telling if the two pairs are the same or not 
print('train_classes size =', train_classes.size())
print('test_input size =', test_input.size())
print('test_target size =', test_target.size())
print('test_classes size =', test_classes.size())

train_input size = torch.Size([1000, 2, 14, 14])
train_target size = torch.Size([1000])
train_classes size = torch.Size([1000, 2])
test_input size = torch.Size([1000, 2, 14, 14])
test_target size = torch.Size([1000])
test_classes size = torch.Size([1000, 2])


### Architectures

In [42]:
class Net_sharing(nn.Module):
    def __init__(self, hidden, act_fun):
        super(Net_sharing, self).__init__()
        self.act_fun = act_fun
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        
        # After concatenation of the features from image 1 and image 2
        self.fc1 = nn.Linear(512, hidden)
        self.fc2 = nn.Linear(hidden,2)

    def forward(self, x):
        conv_images = []
        for image in range(2):
            first_conv = self.act_fun(F.max_pool2d(self.conv1(x[:,image,:,:].view(100,1,14,14)), kernel_size=2, stride=2))
            conv_images.append(self.act_fun(F.max_pool2d(self.conv2(first_conv), kernel_size=2, stride=2)))
        
        x = torch.cat([conv_images[0], conv_images[1]],1)
        x = self.act_fun(self.fc1(x.view(-1, 512)))
        x = self.fc2(x)
        return x


class Net_NOsharing(nn.Module):
    def __init__(self, hidden, act_fun):
        super(Net_NOsharing, self).__init__()
        self.act_fun = act_fun
        self.conv1_1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv1_2 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2_1 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_2 = nn.Conv2d(32, 64, kernel_size=3)
        
        # After concatenation of the features from image 1 and image 2
        self.fc1 = nn.Linear(512, hidden)
        self.fc2 = nn.Linear(hidden,2)

    def forward(self, x):
        x1_1 = self.act_fun(F.max_pool2d(self.conv1_1(x[:,0,:,:].view(100,1,14,14)), kernel_size=2, stride=2))
        x2_1 = self.act_fun(F.max_pool2d(self.conv2_1(x1_1), kernel_size=2, stride=2))
        
        x1_2 = self.act_fun(F.max_pool2d(self.conv1_2(x[:,1,:,:].view(100,1,14,14)), kernel_size=2, stride=2))
        x2_2 = self.act_fun(F.max_pool2d(self.conv2_2(x1_2), kernel_size=2, stride=2))
        
        x = torch.cat([x2_1, x2_2],1)
        x = self.act_fun(self.fc1(x.view(-1, 512)))
        x = self.fc2(x)
        return x

In [43]:
hidden_units = 10
nb_epochs = 10
model_sharing, criterion = Net_sharing(hidden = hidden_units, act_fun = F.relu), nn.CrossEntropyLoss()
model_NOsharing, criterion = Net_NOsharing(hidden = hidden_units, act_fun = F.relu), nn.CrossEntropyLoss()
eta, mini_batch_size = 1e-1, 100
optimizer = torch.optim.Adam(model.parameters(), lr = eta)

In [50]:
def train_model(model, train_input, train_target,nb_epochs, batch_size,criterion,optimizer):    
    for e in range(nb_epochs):
        for step_ in range(0,train_input.size(0),batch_size):                              
            output = model(train_input[step_:step_+batch_size])
            loss = criterion(output, train_target[step_:step_+batch_size])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [51]:
train_model(model_sharing, train_input, train_target,nb_epochs, mini_batch_size,criterion,optimizer)
train_model(model_NOsharing, train_input, train_target,nb_epochs, mini_batch_size,criterion,optimizer)

In [48]:
activation_fcts = ['relu','tanh','sig']
hidden_units = [50, 100, 200]